In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
import time
import random
import unicornhat as uh
import html5lib
from sys import version_info

In [2]:
uh.set_layout(uh.AUTO)
uh.brightness(0.45)
x,y = uh.get_shape()
x_mid,y_mid = uh.get_shape()[0] // 2, uh.get_shape()[1] // 2

def set_hline(col,r,g,b):
        line_ = [(xi,yi,r,g,b) for xi in range(x) for yi in col]
        [uh.set_pixel(line_[i][0],line_[i][1],line_[i][2],line_[i][3],line_[i][4]) for i in range(len(line_))]
        uh.show()
        
def set_hpline(col,rows,r,g,b):
        line_ = [(xi,yi,r,g,b) for xi in range(rows) for yi in col]
        [uh.set_pixel(line_[i][0],line_[i][1],line_[i][2],line_[i][3],line_[i][4]) for i in range(len(line_))]
        uh.show()

def set_vline(row,r,g,b):
        line_ = [(xi,yi,r,g,b) for yi in range(y) for xi in row]
        [uh.set_pixel(line_[i][0],line_[i][1],line_[i][2],line_[i][3],line_[i][4]) for i in range(len(line_))]
        uh.show()
        
def set_boarder(r,g,b):
        x,y = uh.get_shape()
        set_hline([0,y-1],r,g,b)
        set_vline([0,x-1],r,g,b)
        time.sleep(.1)
        
def random_start():
        return (random.randint(0,x-1),random.randint(0,y-1))

def random_color():
        r = random.randint(50,250)
        g = random.randint(50,250)
        b = random.randint(50,250)
        time.sleep(.02)
        return (r,g,b)

def color_inc(r,g,b):
        r = (r + random.randint(5,30)) % 255
        g = (g + random.randint(5,30)) % 255
        b = (b + random.randint(5,30)) % 255
        return (r,g,b)

def color_dec(r,g,b):
        r = (r - random.randint(5,30)) % 255
        g = (g - random.randint(5,30)) % 255
        b = (b - random.randint(5,30)) % 255
        return (r,g,b)
def color_chg(r,g,b):
    rnd_ = random.randint(-1,1)

    if rnd_ == -1:
            return color_dec(r,g,b)
    elif rnd_ == 1:
            return color_inc(r,g,b)
    else:
            return (r,g,b)

def colorinc_r(r,g,b,magnitude): #magnitude is % change ie: 5 -> 5% increase
    try:
        return (int(r* (1 + magnitude/100) % 255),g,b)
    except:
        print ('Magnitude error!')

def colorinc_g(r,g,b,magnitude):
    try:
        return (r,int(g*(1 + magnitude/100)% 255),b)
    except:
        print ('Magnitude error!')
        
def colorinc_b(r,g,b,magnitude):
    try:
        return (r,g,int(b*(1 + magnitude/100) % 255))
    except:
        print ('Magnitude error!')
        
def colordec_r(r,g,b,magnitude):
    try:
        return (int(r*(1 - magnitude/100) % 255),g,b)
    except:
        print ('Magnitude error!')
        
def colordec_g(r,g,b,magnitude):
    try:
        return (r,int(g*(1 - magnitude/100) % 255),b)
    except:
        print ('Magnitude error!')
        
def colordec_b(r,g,b,magnitude):
    try:
        return (r,g,int(b*(1 - magnitude/100) % 255))
    except:
        print ('Magnitude error!')
        
def RGB_chg(RGBcomponent,magnitude):
    try:
        return max(int(RGBcomponent * (1 + magnitude/100) % 255),110)
    except:
        print ('Magnitude error! \n-\n or RGB error!')
            
        

def set_scale_hline(col,r_start,g_start,b_start,magnitude_r,magnitude_g,magnitude_b):
    r_ = r_start
    g_ = g_start
    b_ = b_start
    for i in range(uh.get_shape()[1]):
        uh.set_pixel(col,i,r_,g_,b_)
        uh.show()
        time.sleep(.2)
        r_ = RGB_chg(r_,magnitude_r)
        g_ = RGB_chg(g_,magnitude_g)
        b_ = RGB_chg(b_,magnitude_b)  
    
def set_scale_vline(row,r_start,g_start,b_start,magnitude_r,magnitude_g,magnitude_b):
    r_ = r_start
    g_ = g_start
    b_ = b_start
    for i in range(uh.get_shape()[0]):
        uh.set_pixel(i,row,r_,g_,b_)
        uh.show()
        time.sleep(.2)
        r_ = RGB_chg(r_,magnitude_r)
        g_ = RGB_chg(g_,magnitude_g)
        b_ = RGB_chg(b_,magnitude_b)            
        
def set_c_boarder(r,g,b):
    x,y = uh.get_shape()
    set_hline([0,y-1],r,g,b)
    set_vline([0,x-1],r,g,b)
    time.sleep(.1)

def random_gridfill():
    RGB=random_color()
    set_scale_vline(0,RGB[0],RGB[1],RGB[2],25,0,0)
    set_scale_vline(1,RGB[0],RGB[2],RGB[1],25,25,0)
    set_scale_vline(2,RGB[1],RGB[0],RGB[2],25,25,25)
    set_scale_vline(3,RGB[1],RGB[2],RGB[0],50,25,25)

            
def uh_reset():
    uh.clear()
    uh.show()


In [3]:
def weather_url():
    py3 = version_info[0] > 2 #creates boolean value for test that Python major version > 2
    if py3:
        Zip_Code = input("Please Enter a Zip Code: ").strip().replace(' ','').split('-')[0]
        assert len(Zip_Code) == 5
        assert Zip_Code.isnumeric()
    else:
        print('Wrong version of Python!\nTry again with Python 3.x')
    url = 'https://weather.com/weather/hourbyhour/l/%s:4:US' %(Zip_Code)
    return(url)

In [4]:
def get_weather_df(zip_code):
    Zip_Code = zip_code.strip().replace(' ','').split('-')[0]
    if len(Zip_Code) == 5 and Zip_Code.isnumeric():
        url = 'https://weather.com/weather/hourbyhour/l/%s:4:US' %(Zip_Code)
    else:
        url = weather_url()

    temp_df = pd.read_html(url)[0]
    temp_df = temp_df.iloc[:,1:]
    temp_df.columns = ['Time','Description','Temp','Feels','Precip','Humidity','Wind']
    temp_df['Time'] = pd.to_datetime(temp_df['Time'])
    temp_df['Temp'] = temp_df['Temp'].map(lambda x: int(x[:-1]))
    temp_df['Feels'] = temp_df['Feels'].map(lambda x: int(x[:-1]))
    temp_df['Precip'] = temp_df['Precip'].map(lambda x: int(x[:-1]))
    temp_df['Humidity'] = temp_df['Humidity'].map(lambda x: int(x[:-1]))
    temp_df['Wind Direction'] = temp_df['Wind'].map(lambda x: x.split()[0])
    temp_df['Wind Speed'] = temp_df['Wind'].map(lambda x: x.split()[1])
    temp_df.index.name=''
    temp_df.index=(temp_df['Time'])
    temp_df = temp_df[['Description','Temp','Feels','Precip','Humidity','Wind Direction','Wind Speed']]
    return (temp_df)

In [5]:

temp_df = get_weather_df('33018')
current_weather = temp_df.iloc[0]
current_weather




Description       Mostly Cloudy
Temp                         83
Feels                        93
Precip                       15
Humidity                     86
Wind Direction              ENE
Wind Speed                    5
Name: 2017-09-04 23:30:00, dtype: object

In [177]:
current_weather = temp_df.iloc[0]
current_weather

Description       Clear
Temp                 72
Feels                72
Precip                0
Humidity             62
Wind Direction      SSW
Wind Speed            6
Name: 10:00 pmMon, dtype: object

In [6]:
#weather func
#temp-bar
def set_temp(temp):
    temp = int(temp)
    def linshuffle(linspace,temp):
        ticks = linspace
        tmp = temp
        ticks.put(0,int(tmp))
        ticks.sort()
        ticks = pd.Series(ticks)
        return (ticks[ticks == temp].index[0])
        
    if int(temp) <= 32:
        T = linshuffle(np.linspace(0,32,8),temp)
        L = int(np.ceil(temp * 8/32))
        R,G,B = (0,0,100)
        set_hpline([0],L,R,G,B)
    elif int(temp) < 75:
        T = linshuffle(np.linspace(32,75,8),temp)
        L = int(np.ceil(temp * 8 /75))
        R,G,B = (0,int(np.ceil(3*int(temp))),0)
        set_hpline([0],L,R,G,B)
    elif int(temp) >= 75:
        T = linshuffle(np.linspace(75,100,8),temp)
        L = int(np.ceil(temp * 1/6) - 9)
        R,G,B = (2*int(temp),0,0)
        set_hpline([0],L,R,G,B)
    
#humidity-bar
def set_humidity(hum):
    humidity = int(hum)
    L = min(int(np.ceil(humidity * 8 / 100)), 8)
    R,G,B = (0,85,max(2*humidity,175))
    set_hpline([1],L,R,G,B)
    
#precipitation-bar
def set_precipitation(precip):
    precipitation = int(precip)
    L = min(int(np.ceil(precipitation * 8 / 100)), 8)
    R,G,B = (0,0,min(int(np.ceil(12 * precipitation)),200))
    #L = int(L)
    set_hpline([2],L,R,G,B)
    
#windspeed-bar
def set_windspeed(wspeed):
    wind_speed = int(wspeed)
    L = min(int(np.ceil(np.log2(wind_speed))), 8)
    R,G,B = (100,100,100)
    set_hpline([3],L,R,G,B)
    


In [7]:
def weatherloop(zip_code,n):
    """
    n defaults to 20,  zip_code needs valid zip code
    """

    uh_reset()
    if n.isnumeric():
        n = int(n)
    else:
        n = 20
    Zip_Code = zip_code.strip().replace(' ','').split('-')[0]
    for i in range(int(n)):
        if len(Zip_Code) == 5 and Zip_Code.isnumeric():
            temp_df = get_weather_df(Zip_Code)
        else:
            temp_df = get_weather_df('07020')
        #temp_df = get_weather_df(Zip_Code)
        current_weather = temp_df.iloc[0]
        set_temp(current_weather['Temp'])
        set_humidity(current_weather['Humidity'])
        set_precipitation(current_weather['Precip'])
        set_windspeed(current_weather['Wind Speed'])
        time.sleep(5)

def weatherset(zip_code):
    Zip_Code = zip_code
    if len(Zip_Code) == 5 and Zip_Code.isnumeric():
        temp_df = get_weather_df(Zip_Code)
    else:
        temp_df = get_weather_df('07020')
    #temp_df = get_weather_df(Zip_Code)
    current_weather = temp_df.iloc[0]
    uh_reset()
    set_temp(current_weather['Temp'])
    set_humidity(current_weather['Humidity'])
    set_precipitation(current_weather['Precip'])
    set_windspeed(current_weather['Wind Speed'])
    
    

In [8]:
weatherset('33018')

In [9]:
weatherloop('07020','10')



In [11]:
uh_reset()



In [ ]:
market = pd.read_html('http://www.nasdaq.com/markets/commodities.aspx')[4]

In [ ]:
market.columns = ['Index','Last','Change %']
market

In [ ]:
def chgamt(x):
    val = x
    val_ = float(val.split()[0])
    if val.split()[1] == str(chr(9650)):
        return (val_)
    elif val.split()[1] == str(chr(9660)):
        return (-1 * val_)

In [ ]:
market['Last'] = market['Last'].astype(float)
market['Change Value'] = market['Change %'].map(chgamt)
market['Change Percent'] = market['Change %'].map(lambda x: float(x.split()[2][:-1]))
market[['Index','Last','Change Value', 'Change Percent']]

In [12]:
t = pd.read_html('http://www.phaster.com/zip_code.html')
t=t[0]
t.columns = t.iloc[0,:]
print (t.columns)
t = t.iloc[1:,:]
t['First Zip'] = t['ZIP Code(s)'].map(lambda x: x.replace('-',' ').split()[0])
t[['State','City','First Zip']]


Index(['State', 'City', 'ZIP Code(s)'], dtype='object')


,State,City,First Zip
1,Alabama (AL),Huntsville,35801
2,Alaska (AK),Anchorage,99501
3,Arizona (AZ),Phoenix,85001
4,Arkansas (AR),Little Rock,72201
5,California (CA),Sacramento Los Angeles Beverly Hills,94203
6,Colorado (CO),Denver,80201
7,Conneticut (CT),Hartford,06101
8,Deleware (DE),Dover,19901
9,District of Columbia (DC),Washington,20001
10,Florida (FL),Pensacola Miami Orlando,32501


In [15]:
Z = t['First Zip'].tolist()
for z in Z:
    try:
        weatherset(z)
    except:
        print("Something went wrong for %s" %(z))

Something went wrong for 35801
Something went wrong for 02101
Something went wrong for 49036
Something went wrong for 87500
Something went wrong for 29020
Something went wrong for 78701
